<a href="https://colab.research.google.com/github/BrunoReis136/tensorflow/blob/main/TensorFlowMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MNIST COM CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def model_mnist():
  (x_train, y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

  x_train = x_train[..., tf.newaxis].astype("float32") / 255.0
  x_test = x_test[..., tf.newaxis].astype("float32") / 255.0

  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D((2,2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1
  )

  datagen.fit(x_train)

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

  model.compile(optimizer="adam",
                loss=loss_fn,
                metrics=["accuracy"])

  prediction_single_sample = model(x_train[:1])

  probabilities_single_sample = tf.nn.softmax(prediction_single_sample).numpy()
  print(f'Previsões previstas para a primeira amostra: {probabilities_single_sample}')

  loss_single_sample = loss_fn(y_train[:1],prediction_single_sample).numpy()
  print(f'Perda para a primeira amostra: {loss_single_sample}')

  class myCallback(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.final_acc = None

    def on_epoch_end(self, epoch, logs=None):
      logs = logs or {}
      acc = logs.get('accuracy')

      if acc is not None and acc > 0.995 :
          print('\nValores de Precisão ou Perda atingidos.')
          self.final_acc = acc
          self.model.stop_training = True

  callback = myCallback()

  print('Iniciando treinamento do modelo...')
  model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=20, callbacks=[callback])

  print('Avaliando modelo...')
  model.evaluate(x_test, y_test, verbose=2)

  salva_sn = input('Deseja salvar o modelo treinado?(s/n)').lower().strip()

  if salva_sn == 's':
    if callback.final_acc is not None:
      label = round(callback.final_acc,3)*100
    else:
      label = 'test'

    modelo_savepath = f'model_cnn({label}).keras'

    try:
      model.save(modelo_savepath)
      print('Modelo salvo com csucesso.')
    except Exception as e:
      print(f'Erro ao salvar modelo: {e}')
  else:
    print('Modelo não salvo.')

model_mnist()


<h1>TESTE PRÁTICO COM WIDGET INPUTS</h1>




In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from ipywidgets import FileUpload, Text, Button, Output, VBox, HBox
from IPython.display import display, Image
from io import BytesIO
from PIL import Image as PILImage # Para manipular a imagem
import matplotlib.pyplot as plt

model_mnist = tf.keras.models.load_model('model_cnn(test).keras')

uploader = FileUpload(
    accept='image/*',
    multiple=False,
    description='Carregar Imagem'
)

label_input = Text(
    placeholder='Digite a label correta (dígito de 0 a 9)',
    description='Label Correta:',
    disabled=False
)

test_button = Button(
    description='Testar Modelo',
    button_style='success',
    tooltip='Clique para testar o modelo com a imagem e label fornecidas'
)

output_area = Output()

def preprocess_image(image_bytes):
    img = PILImage.open(BytesIO(image_bytes)).convert('L')  # escala de cinza
    img = img.resize((28, 28))  # redimensionar
    img_array = np.array(img) / 255.0  # normalizar
    img_array = np.expand_dims(img_array, axis=-1)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array.astype(np.float32)

class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


def on_test_button_clicked(b):
    with output_area:
        output_area.clear_output()

        if not uploader.value:
            print("Por favor, carregue uma imagem primeiro.")
            return

        if not label_input.value.isdigit() or int(label_input.value) not in range(10):
          print("Por favor, insira uma label numérica entre 0 e 9.")
          return

        uploaded_filename = list(uploader.value.keys())[0]
        image_content = uploader.value[uploaded_filename]['content']

        try:

            processed_image = preprocess_image(image_content)
            model_mnist(processed_image)
            display(Image(data=image_content, width=200))

            predictions = model_mnist.predict(processed_image)
            probabilities = tf.nn.softmax(predictions[0]).numpy()
            predicted_class_index = np.argmax(probabilities)
            predicted_probability = np.max(probabilities)
            predicted_class_name = class_names[predicted_class_index]

            true_label = label_input.value.strip().lower()
            predicted_label = predicted_class_name.lower()

            print(f"Previsão do Modelo: {predicted_class_name} (Probabilidade: {predicted_probability:.2f})")
            print(f"Label Correta Informada: {true_label}")

            if predicted_label == true_label:
              print("Resultado: ✅ O modelo reconheceu corretamente!")
            else:
              print(f"Resultado: ❌ O modelo NÃO reconheceu corretamente. Ele previu '{predicted_class_name}' mas a label correta era '{true_label}'.")

        except Exception as e:
            print(f"Ocorreu um erro: {e}")
            print("Verifique se o pré-processamento da imagem está correto para o seu modelo (tamanho, canais, normalização).")


test_button.on_click(on_test_button_clicked)

input_widgets = VBox([uploader, label_input, test_button])
display(VBox([input_widgets, output_area]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

image = Image.open("JS.png")

plt.grid = False
plt.gray()
plt.axis('off')
plt.imshow(image)
plt.show()

image_transformed = np.copy(image.convert('L'))

size_x = image_transformed.shape[0]
size_y = image_transformed.shape[1]

print(f'Tamanho da imagem: {size_x}px por {size_y}px')

image = np.array(image.convert('L')).astype(float)

filter = [[1, 0, -1],
          [1, 0, -1],
          [1, 0, -1]]

weight = 1

for x in range(1, size_x-1):
  for y in range(1, size_y-1):
    convolution = 0.0
    convolution = convolution + (image[x-1, y-1] * filter[0][0])
    convolution = convolution + (image[x-1, y] * filter[0][1])
    convolution = convolution + (image[x-1, y+1] * filter[0][2])
    convolution = convolution + (image[x, y-1] * filter[1][0])
    convolution = convolution + (image[x, y] * filter[1][1])
    convolution = convolution + (image[x, y+1] * filter[1][2])
    convolution = convolution + (image[x+1, y-1] * filter[2][0])
    convolution = convolution + (image[x+1, y] * filter[2][1])
    convolution = convolution + (image[x+1, y+1] * filter[2][2])

    convolution = convolution + weight

    if convolution < 0:
      convolution = 1
    if convolution > 255:
      convolution = 255

    image_transformed[x,y] = convolution

plt.gray()
plt.grid =  False
plt.imshow(image_transformed)
plt.show()


new_x = int(size_x /2)
new_y = int(size_y /2)

newImage = np.zeros((new_x, new_y))

for x in range(0, size_x , 2):
  for y in range(0, size_y, 2):
    pixels = []
    pixels.append(image_transformed[x,y])
    pixels.append(image_transformed[x+1,y])
    pixels.append(image_transformed[x,y+1])
    pixels.append(image_transformed[x+1,y+1])

    newImage[int(x/2), int(y/2)] = max(pixels)

plt.gray()
plt.grid = False
plt.imshow(newImage)
plt.show()





In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train, x_test = x_train[..., tf.newaxis].astype("float32") / 255,x_test[..., tf.newaxis].astype("float32") / 255

datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

datagen.fit(x_train)

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape = (28, 28, 1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10,activation='softmax')
])

class callBack(tf.keras.callbacks.Callback):
  def __init__(self):
    super().__init__()
    self.final_acc = None
  def on_epoch_end(self, epoch, logs = None):
    logs = logs or {}
    acc = logs.get('accuracy')

    if acc is not None and acc > 0.99:
      self.final_acc = acc
      self.model.stop_training = True

callback = callBack()


model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs = 10,
    callbacks = [callback]
)

model.evaluate(x_test, y_test, verbose = 2)


In [ ]:
import tensorflow as tf
import zipfile

with zipfile.ZipFile('images.zip','r') as zip_ref:
  zip_ref.extractall('images')

ds_train = tf.keras.utils.image_dataset_from_directory(
    "images/images/training",
    image_size=(28,28),
    batch_size=32,
    label_mode = 'binary'
)

ds_valid = tf.keras.utils.image_dataset_from_directory(
    'images/images/validation',
    image_size = (300,300),
    batch_size = 32,
    label_mode = 'binary'
)

rescale_layer = tf.keras.layers.Rescaling(scale=1./255)

ds_train_rescaled = ds_train.map(lambda image, label:(rescale_layer(image), label))
ds_valid_rescaled = ds_valid.map(lambda image, label:(rescale_layer(image), label))

ds_train_final = (ds_train_rescaled.cache().shuffle(buffer_size = 1000).prefetch(buffer_size = tf.data.AUTOTUNE))
ds_valid_final = (ds_valid_rescaled.cache().prefetch(buffer_size = tf.data.AUTOTUNE))

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28,28,3)),
    tf.keras.layers.Conv2D(16,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.Dente(512,activation='relu'),
    tf.keras.layers.Dropout(0.2)
    tf.keras.layers.Dense(1, activation='sigmoid')

model.compile(
    loss='binary_crossentropy',
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001),
    metric = 'accuracy'
)

history = model.fit(
    ds_train_final,
    epochs = 15,
    validation_data = ds_valid_final,
    verbose=2
)

])


In [ ]:
import os
import random
import numpy as np
from io import BytesIO
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf

from ipywidgets import widgets


with zipfile.ZipFile('horse-or-human-train.zip','r') as file_train:
  file_train.extractall('horse-or-human-train')

with zipfile.ZipFile('horse-or-human-validate.zip','r') as file_validate:
  file_validate.extractall('horse-or-human-validate')


datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

ds_train = datagen.flow_from_directory(
    'horse-or-human-train',
    target_size = (300,300),
    batch_size = 32,
    class_mode = 'binary'
)

val_datagen = ImageDataGenerator(rescale=1./255)

ds_validate = val_datagen.flow_from_directory(
    'horse-or-human-validate',
    target_size=(300,300),
    batch_size=32,
    class_mode='binary'
)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(300,300,3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

model = tf.keras.models.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1,activation='sigmoid')
])


earlyStop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

class callBack(tf.keras.callbacks.Callback):
  def __init__(self):
    super().__init__()
  def on_epoch_end(self, epoch, logs):
    acc = logs['accuracy'] or None

    if acc > 0.99:
      self.model.stop_training = True

callback = callBack()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(ds_train,
          epochs=10,
          verbose=1,
          callbacks=[earlyStop])

model.evaluate(ds_validate, verbose=1)

model.save('horse-or-human-model.keras')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


/tmp/ipython-input-35-2378972655.py:51: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 115s 3s/step - accuracy: 0.8546 - loss: 0.2437
Epoch 2/10


/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 106s 3s/step - accuracy: 0.9999 - loss: 0.0027
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 107s 3s/step - accuracy: 0.9989 - loss: 0.0038
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 106s 3s/step - accuracy: 0.9960 - loss: 0.0076
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - accuracy: 1.0000 - loss: 0.0012
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 146s 3s/step - accuracy: 1.0000 - loss: 7.3036e-04
Epoch 7/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 139s 3s/step - accuracy: 1.0000 - loss: 5.6562e-04
Epoch 8/10
19/33 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 1.0000 - loss: 2.6854e-04

<h1>TESTE DE IMAGEM HORSE-OR-HUMAN</h1>

In [34]:
from google.colab import files
import numpy as np

uploader = files.upload()
model = tf.keras.models.load_model('horse-or-human-model.keras')

for filename in uploader.keys():
  path = '/content/' + filename
  image = tf.keras.utils.load_img(path, target_size=(300,300))
  image = tf.keras.utils.img_to_array(image)
  image = np.expand_dims(image, axis=0)

  prediction = model.predict(image, verbose=0)[0][0]
  print(f'nmodel output: {prediction}')

  if prediction > 0.5:
    print(filename +  ' é um humano.')
  else:
    print(filename + ' é um cavalo.')


Saving 1.png to 1 (2).png
nmodel output: 0.596562922000885
1 (2).png é um humano.


In [ ]:
import ipywidgets as widget
from IPython.display import display, Image
from PIL import Image as PILImage
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('horse-or-human-model.keras')

image_up = FileUpload(accept='/image/*', multiple=False, description='Carregar Imagem')
btn = Button(
    description='Enviar imagem para o teste',
    button_style='success',
    tooltip='Clique para testar o modelo com a imagem e label fornecidas'
)

display(image)

def testar_imagem():
  with output_area:
    output_area.clear_output()
    classes = ['cavalo','humano']
    previsao = model.predict(image)
    print(previsao)
    possibilidade = tf.nn.softmax(previsao[0]).numpy()
    classe_provavel_index = np.argmax(possibilidade)
    classe_provavel = classes[classe_provavel_index]

    print(f'Classe provavel: {classe_provavel}')

btn.on_click(testar_imagem())


input_widgets = VBox([image_up, btn])
output_area = Output()
display(VBox([input_widgets, output_area]))

In [21]:
import os
import random
import numpy as np
from io import BytesIO

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf

from ipywidgets import widgets

TRAIN_DIR = 'horse-or-human-train'

print(f'Files in current directory: {od.listdir()}')

print(f'\n subdirectories within {TRAIN_DIR} : dir: {os.listdir(TRAIN_DIR)}')


train_horses_dir = os.path.join(TRAIN_DIR, 'horses')
train_human_dir = os.path.join(TRAIN_DIR, 'human')

train_horses_names = os.listdir(train_horses_dir)
print(train_horses_names[0:5])
train_human_names = os.listdir(train_human_dir)
print(train_human_names[0:5])

print(f'O tamanho da lista de nomes de cavalos é de: {len(os.listdir(train_horses_dir))}')
print(f'O tamanho da lista de nomes de humanos é de: {len(os.listdir(train_human_dir))}')


nrows = 4
ncols = 4

fig = plt.gcf()
fig.set.size_inches(ncols=3, nrows=3)

next_horse_pix = [os.path.join(train_horses_dir, fname)
                  for fname in random.sample(train_horses_names, k=8)]
next_human_pix = [os.path.join(train_human_dir, fname)
                  for fname in random.sample(train_horses_names, k=8)]

for i, img_path in enumerate(next_horse_pix + next_human_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off')

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

ModuleNotFoundError: No module named 'ipywidget'